In [18]:
import sys
from tqdm import tqdm
sys.path.insert(0, '../')

import torch
from torch import nn
from torch.nn import functional as F

from attention import MultiHeadAttention
from encoder import Encoder

In [19]:
D_MODEL = 6
NUM_HEADS = 2
MAX_LEN = 4
encoder = Encoder(d_model=6, num_heads=2, max_len=4)

In [21]:
ln = nn.LayerNorm(normalized_shape=D_MODEL)

In [22]:
attn_scores = encoder(tensor_rand)

torch.Size([4, 1, 6]) torch.Size([4, 1, 6])
Error
..\attention.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.matmul(query, key.transpose(-1, -2)) / math.sqrt(self.d_model)


In [24]:
ln(attn_scores).size()

torch.Size([4, 1, 6])

In [7]:
attn_scores

tensor([[[ 1.0609, -0.0602, -0.0560,  0.7548,  0.6464,  0.9860]],

        [[ 0.3296, -0.0277,  0.3275,  0.3069,  0.8603,  1.0444]],

        [[ 0.6354,  0.2358,  0.5755,  0.8440,  0.4686,  0.7855]],

        [[ 0.1682,  0.4743,  0.2788,  0.1155,  0.6572,  0.3863]]],
       grad_fn=<AsStridedBackward>)

In [7]:
attn_scores.size()

torch.Size([4, 1, 4])

In [9]:
residual.size()

torch.Size([4, 4])

In [24]:
attn = torch.rand(4,4)
attn

tensor([[0.4888, 0.3735, 0.3462, 0.1984],
        [0.4878, 0.9013, 0.0998, 0.2447],
        [0.7737, 0.5993, 0.8875, 0.4567],
        [0.7347, 0.8924, 0.2499, 0.1155]])

In [31]:
float('-inf')

-inf

In [27]:
torch.masked_fill(mask=mask, value=0)

TypeError: masked_fill() received an invalid combination of arguments - got (mask=Tensor, value=int, ), but expected one of:
 * (Tensor input, Tensor mask, Tensor value)
 * (Tensor input, Tensor mask, Number value)


In [25]:
mask = torch.tril(torch.ones(4,4), diagonal=0).bool()

In [12]:
pad_id = 0
vocab_size = 100
max_len = 10
hidden_dim = 6

def padding(data: list, pad_id: int=0) -> (list, int):
    max_len = len(max(data, key=len))
    output = [sample + [pad_id]*(max_len-len(sample)) for sample in tqdm(data)]
    return output, max_len

data = [
     [62, 13, 47, 39, 78, 33, 56, 13, 39, 29],
     [60, 96, 51, 32, 90, 44, 86, 71, 36, 18],
     [35, 45, 48, 65, 91, 99, 92, 10, 31, 21],
     [75, 51, 45, 48, 65, 91, 99, 11, 13, 28],
     [66, 88, 98, 47, 48, 65, 17, 13, 67, 99],
     ]

# data, max_len = padding(data)
data = torch.LongTensor(data)
print(data.shape)

torch.Size([5, 10])


In [13]:
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=hidden_dim)
embedding.weight.data.uniform_(-1, 1)
print('Weight initialized')

Weight initialized


In [14]:
X_embedded = embedding(data)
X_embedded.size()

torch.Size([5, 10, 6])

In [15]:
w_query = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Query
w_key = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Key
w_value = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Value

query = w_query(X_embedded)
key = w_key(X_embedded)
value = w_value(X_embedded)

In [19]:
attention = MultiHeadAttention(hidden_dim=hidden_dim, num_heads=2)
attention_mat = attention(query, key, value)

In [21]:
attention_mat.size()

torch.Size([5, 10, 6])

In [26]:
attn_layer_norm = nn.LayerNorm(normalized_shape=(10, 6))

In [25]:
attention_mat.size()[1:]

torch.Size([10, 6])

In [27]:
attn_layer_norm(attention_mat).size()

torch.Size([5, 10, 6])